In [1]:
import tensorflow as tf
from tensorflow import keras
import joblib
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import cv2
from server.Wavelet import wavelett_trans

In [2]:
celeb_folders = joblib.load('celeb_folders')

In [3]:
encode_celeb = {}
c = 0

for name in celeb_folders:
    encode_celeb[name] = c
    c += 1
encode_celeb

{'ben_afflek': 0,
 'jerry_seinfeld': 1,
 'lionel_messi': 2,
 'madonna': 3,
 'maria_sharapova': 4,
 'mindy_kaling': 5,
 'roger_federer': 6,
 'serena_williams': 7,
 'virat_kohli': 8}

In [4]:
X = []
Y = []
for celeb, path in celeb_folders.items():
    if not path:
        continue
    for train_data in path:
        img_train = cv2.imread(train_data)
        if img_train is None:
            continue
        scaled_raw_img = cv2.resize(img_train, (32,32))
        img_har = wavelett_trans(img_train, level=5)
        scaled_har_img = cv2.resize(img_har, (32,32))
        stack_img = np.vstack((scaled_raw_img.reshape(32*32*3,1), scaled_har_img.reshape(32*32,1)))
        stack_img = stack_img.reshape(64,64)
        X.append(stack_img)
        Y.append(encode_celeb[celeb])

In [5]:
X = np.array(X, dtype=np.uint8)
Y = np.array(Y, dtype=np.uint8)

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, random_state=20, test_size=0.1)

In [7]:
X_train = X_train/255
X_test = X_test/255

In [8]:
X_train.shape, X_test.shape

((198, 64, 64), (23, 64, 64))

In [9]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(64,64)),
    keras.layers.Dense(2048, activation='relu'),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.Dense(10, activation='sigmoid')
])
opt = keras.optimizers.Adam(learning_rate=0.00001)
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=30)  

Epoch 1/30
7/7 [==============================] - 4s 128ms/step - loss: 2.2638 - accuracy: 0.1869
Epoch 2/30
7/7 [==============================] - 1s 147ms/step - loss: 1.9860 - accuracy: 0.3485
Epoch 3/30
7/7 [==============================] - 1s 134ms/step - loss: 1.8856 - accuracy: 0.3788
Epoch 4/30
7/7 [==============================] - 1s 146ms/step - loss: 1.7743 - accuracy: 0.4596
Epoch 5/30
7/7 [==============================] - 1s 140ms/step - loss: 1.6902 - accuracy: 0.5455
Epoch 6/30
7/7 [==============================] - 1s 140ms/step - loss: 1.6013 - accuracy: 0.6162
Epoch 7/30
7/7 [==============================] - 1s 121ms/step - loss: 1.5305 - accuracy: 0.6717
Epoch 8/30
7/7 [==============================] - 1s 137ms/step - loss: 1.4490 - accuracy: 0.7020
Epoch 9/30
7/7 [==============================] - 1s 140ms/step - loss: 1.3962 - accuracy: 0.7121
Epoch 10/30
7/7 [==============================] - 1s 140ms/step - loss: 1.3158 - accuracy: 0.7778
Epoch 11/30
7/7 [==

In [10]:
model.evaluate(X, Y)

7/7 [==============================] - 0s 13ms/step - loss: 9.9522 - accuracy: 0.9548


[9.952239036560059, 0.9547511339187622]

In [11]:
model_cnn = keras.Sequential([
    keras.layers.Conv2D(input_shape=(64,64,1) ,filters=32, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    
    keras.layers.Conv2D(filters= 32, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    
    keras.layers.Flatten(),
    keras.layers.Dense(1024, activation='tanh'),
    keras.layers.Dense(256, activation='tanh'),
    keras.layers.Dense(10, activation='softmax')
])
opt = keras.optimizers.Adam(learning_rate=0.00001)
model_cnn.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model_cnn.fit(X_train, Y_train, epochs=20) 

Epoch 1/20
7/7 [==============================] - 2s 118ms/step - loss: 2.2938 - accuracy: 0.0707
Epoch 2/20
7/7 [==============================] - 1s 128ms/step - loss: 2.1740 - accuracy: 0.1465
Epoch 3/20
7/7 [==============================] - 1s 125ms/step - loss: 2.1372 - accuracy: 0.1667
Epoch 4/20
7/7 [==============================] - 1s 123ms/step - loss: 2.1199 - accuracy: 0.1566
Epoch 5/20
7/7 [==============================] - 1s 120ms/step - loss: 2.1014 - accuracy: 0.1869
Epoch 6/20
7/7 [==============================] - 1s 120ms/step - loss: 2.0836 - accuracy: 0.2273
Epoch 7/20
7/7 [==============================] - 1s 120ms/step - loss: 2.0664 - accuracy: 0.1869
Epoch 8/20
7/7 [==============================] - 1s 121ms/step - loss: 2.0571 - accuracy: 0.1869
Epoch 9/20
7/7 [==============================] - 1s 117ms/step - loss: 2.0476 - accuracy: 0.1869
Epoch 10/20
7/7 [==============================] - 1s 125ms/step - loss: 2.0322 - accuracy: 0.1919
Epoch 11/20
7/7 [==

In [12]:
model_cnn.evaluate(X, Y)

7/7 [==============================] - 0s 16ms/step - loss: 2.1543 - accuracy: 0.2715


[2.1543476581573486, 0.27149322628974915]

In [13]:
joblib.dump(model, 'Deep_Model.sav')

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...layers\flatten
......vars
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-02-04 14:07:44         2400
metadata.json                                  2023-02-04 14:07:44           64
variables.h5                                   2023-02-04 14:07:44    126014352


['Deep_Model.sav']